In [10]:
!pip install streamlit pandas

In [11]:
import os

os.makedirs("smartsupply", exist_ok=True)

In [12]:
%%writefile smartsupply/database.py

import sqlite3

class Database:
    def __init__(self, db_name="smartsupply.db"):
        self.conn = sqlite3.connect(db_name, check_same_thread=False)
        self.cursor = self.conn.cursor()
        self.create_tables()

    def create_tables(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS products (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                price REAL,
                stock INTEGER
            )
        """)

        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS sales (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                product_id INTEGER,
                quantity INTEGER,
                total REAL
            )
        """)
        self.conn.commit()

Overwriting smartsupply/database.py


In [13]:
%%writefile smartsupply/inventory.py

class Inventory:
    def __init__(self, db):
        self.db = db

    def add_product(self, name, price, stock):
        self.db.cursor.execute(
            "INSERT INTO products (name, price, stock) VALUES (?, ?, ?)",
            (name, price, stock)
        )
        self.db.conn.commit()

    def get_products(self):
        self.db.cursor.execute("SELECT * FROM products")
        return self.db.cursor.fetchall()

    def update_stock(self, product_id, new_stock):
        self.db.cursor.execute(
            "UPDATE products SET stock=? WHERE id=?",
            (new_stock, product_id)
        )
        self.db.conn.commit()

Overwriting smartsupply/inventory.py


In [14]:
%%writefile smartsupply/sales.py

class Sales:
    def __init__(self, db):
        self.db = db

    def record_sale(self, product_id, quantity):
        self.db.cursor.execute(
            "SELECT price, stock FROM products WHERE id=?",
            (product_id,)
        )
        product = self.db.cursor.fetchone()

        if product:
            price, stock = product
            if stock >= quantity:
                total = price * quantity

                self.db.cursor.execute(
                    "INSERT INTO sales (product_id, quantity, total) VALUES (?, ?, ?)",
                    (product_id, quantity, total)
                )

                self.db.cursor.execute(
                    "UPDATE products SET stock=? WHERE id=?",
                    (stock - quantity, product_id)
                )

                self.db.conn.commit()
                return True
        return False

    def get_sales(self):
        self.db.cursor.execute("SELECT * FROM sales")
        return self.db.cursor.fetchall()

Overwriting smartsupply/sales.py


In [15]:
%%writefile smartsupply/analytics.py

import pandas as pd

class Analytics:
    def __init__(self, db):
        self.db = db

    def sales_report(self):
        query = """
        SELECT p.name, SUM(s.quantity) as total_qty, SUM(s.total) as revenue
        FROM sales s
        JOIN products p ON s.product_id = p.id
        GROUP BY p.name
        """
        df = pd.read_sql(query, self.db.conn)
        return df

Overwriting smartsupply/analytics.py


In [17]:
%%writefile smartsupply/app.py
import sys
sys.path.append("/content/smartsupply")  # Add smartsupply folder to path

import streamlit as st
from database import Database
from inventory import Inventory
from sales import Sales
from analytics import Analytics

db = Database()
inventory = Inventory(db)
sales = Sales(db)
analytics = Analytics(db)

st.set_page_config(page_title="SmartSupply", layout="wide")
st.title("SmartSupply — Inventory & Sales System")

menu = st.sidebar.selectbox("Menu", ["Inventory", "Sales", "Analytics"])

if menu == "Inventory":
    st.header("Inventory Management")

    name = st.text_input("Product Name")
    price = st.number_input("Price", min_value=0.0)
    stock = st.number_input("Stock", min_value=0, step=1)

    if st.button("Add Product"):
        inventory.add_product(name, price, stock)
        st.success("Product added successfully")

    st.subheader("Product List")
    st.table(inventory.get_products())

elif menu == "Sales":
    st.header("Sales")

    products = inventory.get_products()
    product_map = {f"{p[1]} (Stock: {p[3]})": p[0] for p in products}

    if products:
        product_name = st.selectbox("Select Product", list(product_map.keys()))
        quantity = st.number_input("Quantity", min_value=1, step=1)

        if st.button("Record Sale"):
            success = sales.record_sale(product_map[product_name], quantity)
            if success:
                st.success("Sale recorded successfully")
            else:
                st.error("Not enough stock")

        st.subheader("Sales Records")
        st.table(sales.get_sales())
    else:
        st.info("No products available. Add products first.")

elif menu == "Analytics":
    st.header("Business Analytics")

    report = analytics.sales_report()
    st.dataframe(report)

    if not report.empty:
        st.metric("Total Revenue", f"₹ {report['revenue'].sum():.2f}")
    else:
        st.info("No sales data available yet.")

Overwriting smartsupply/app.py
